In [1]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [2]:
class State(TypedDict):
    #Messafes have the type "list". The "add_messages" function in the annotation defines how this state key should be updated (in this case, it appends messages to the list, rather than overwriting them)
    messages:Annotated[list, add_messages]

graph_builder=StateGraph(State)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
#way 1 to initialize llm
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000206F6580230>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000206F65832F0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
#way 2 to initialize llm
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015678972D20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015678973260>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
#Creating the Node for Chatbot
def chatbot(state:State):
    return { "messages": [ llm.invoke(state["messages"])]}

In [6]:
#Building stateGraph for Chatbot 
graph_builder=StateGraph(State)
#Adding Node to the graph
graph_builder.add_node("llmchatbot", chatbot)

#Adding Edge to the graph
graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END)

#Compiling the graph
graph = graph_builder.compile()


In [9]:
#Run the Chatbot
response = graph.invoke({"messages":"Hii, Write ABCD..!"})
response["messages"][-1].content

APIConnectionError: Connection error.